<h1 style="color:blue;">10_sınıflandırma_alıştırma - 1 </h1>

In [1]:
# SosyalMedyaReklamKampanyasi.csv veri setini kullanarak müşteri satın alma kararını tahmin
# eden bir sınıflandırma modeli oluşturunuz

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [3]:
df = pd.read_csv("D:/Datasets/SosyalMedyaReklamKampanyasi.csv")

# 1. Veri keşfi

In [4]:
df.head()

KullaniciID Cinsiyet  Yas  TahminiMaas  SatinAldiMi
0     15624510    Erkek   19        19000            0
1     15810944    Erkek   35        20000            0
2     15668575    Kadın   26        43000            0
3     15603246    Kadın   27        57000            0
4     15804002    Erkek   19        76000            0

In [5]:
df.groupby(['SatinAldiMi']).KullaniciID.agg(['count']).head()

count
SatinAldiMi       
0              257
1              143

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
KullaniciID    400 non-null int64
Cinsiyet       400 non-null object
Yas            400 non-null int64
TahminiMaas    400 non-null int64
SatinAldiMi    400 non-null int64
dtypes: int64(4), object(1)
memory usage: 15.8+ KB


In [7]:
# Boş değer yok

In [8]:
df.describe()

KullaniciID         Yas    TahminiMaas  SatinAldiMi
count  4.000000e+02  400.000000     400.000000   400.000000
mean   1.569154e+07   37.655000   69742.500000     0.357500
std    7.165832e+04   10.482877   34096.960282     0.479864
min    1.556669e+07   18.000000   15000.000000     0.000000
25%    1.562676e+07   29.750000   43000.000000     0.000000
50%    1.569434e+07   37.000000   70000.000000     0.000000
75%    1.575036e+07   46.000000   88000.000000     1.000000
max    1.581524e+07   60.000000  150000.000000     1.000000

In [9]:
# Aykırı değer görünmüyor.

# 2. Veri Temizliği ve Ön Hazırlığı

In [10]:
# Veri önceden temizlenmiş. 
# Bu aşamada birşey yapmıyoruz.

## 2.1. Veriyi X,y olarak bölme (nitelikler matrisi ve hedef değişken - dikey bölme)

In [11]:
# hedef nitelik en sonda. -1' kadar hepsini seçiyoruz (:-1).
X = df.iloc[:,1:-1].values
X.shape

(400, 3)

In [12]:
# Kontrol
X[:3]

array([['Erkek', 19, 19000],
       ['Erkek', 35, 20000],
       ['Kadın', 26, 43000]], dtype=object)

In [13]:
# Hedef nitelik (Species) en sonda (-1)
y = df.iloc[:,-1].values
y.shape

(400,)

In [14]:
# Kontrol
y[:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int64)

## 2.2. Kategorik Nitelik Dönüştürme

In [15]:
# Kategorik nitelik olarak Cinsiyet ve hedef niteik var (y)

## 2.2.1. Cinsiyet için Label Encoder

In [16]:
from sklearn.preprocessing import LabelEncoder
label_encoder_cinsiyet = LabelEncoder()

In [17]:
X[:,0] = label_encoder_cinsiyet.fit_transform(X[:,0])

In [18]:
X[:3]

array([[0, 19, 19000],
       [0, 35, 20000],
       [1, 26, 43000]], dtype=object)

In [19]:
# OneHotEncoder yapmıyoruz. Çünkü binary.

In [20]:
y[:3]

array([0, 0, 0], dtype=int64)

## 2.3. Feature Scaling (Standardizasyon)

In [21]:
# Yapmak zorundayız
from sklearn.preprocessing import StandardScaler

In [22]:
scaler = StandardScaler()

In [23]:
X = scaler.fit_transform(X)

In [24]:
X.shape

(400, 3)

In [25]:
X[:3]

array([[-1.02020406, -1.78179743, -1.49004624],
       [-1.02020406, -0.25358736, -1.46068138],
       [ 0.98019606, -1.11320552, -0.78528968]])

## 2.4. Veriyi eğitim ve test setlerine bölme (yatay bölme)

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=142)

In [28]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(300, 3)
(100, 3)
(300,)
(100,)


# 3. Model Oluşturma ve Eğitim

In [29]:
from sklearn.neighbors import KNeighborsClassifier

In [30]:
classifier = KNeighborsClassifier(n_neighbors=5)

In [31]:
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [32]:
# minkowski: mesafe metriği
# n_neighbors: kaç komşuya bakılıp kararverilecek, tek sayı olmalı
# p: eğer minkowski seçersek hangi yöntemi olsun
# hiperparametre açıklamaları:
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier

# 4. Predictions

In [33]:
y_pred = classifier.predict(X_test)

## 4.1. Tahmin sonuçlarını numpy.ndarray'den dataframe yapma (İncelemek daha Kolay)

In [34]:
predictions = pd.DataFrame(data=y_pred,    # values
                index=range(len(y_pred)),    # 1st column as index
                   columns=['y_pred'])  # 1st row as the column names

# Sadece y_pred'den oluşan df'e test(gerçek) y_test'i sütun olarak ekleme
predictions['y_test'] = y_test
predictions.head()

y_pred  y_test
0       1       1
1       1       1
2       0       0
3       1       0
4       1       0

# 5. Model Değerlendirme

## 5.1. Hata Matrisi (Confusion Matrix)

In [35]:
from sklearn.metrics import confusion_matrix

In [36]:
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(cm)

[[58  9]
 [ 5 28]]


In [37]:
from sklearn.metrics import accuracy_score

In [38]:
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: % {:10.2f}".format(accuracy*100)) 
# :10.2f ifadesi ondalık kısımda iki basamak göstermek için

Accuracy: %      86.00


## 5.3. En iyi k değeri

In [39]:
# Üretilecek accuracy değerlerini toplayacağımız liste
accuracy_list = []

# k değerlerinin toplanacağı liste
k_list = []

# Arzu edilen k değerleri boyunca döngü çalıştır ve her bir k değeri için
# elde edilen accuracy skorlarını ve k değerlerini birer listede topla
for k in range(2,10):
    #sınıflandırıcı küme sayısı olarak k alıyor
    classifier = KNeighborsClassifier(n_neighbors=k, metric='minkowski', p=2)
    # Eğitim
    classifier.fit(X_train, y_train)
    # Tahmin
    y_pred = classifier.predict(X_test)
    # Accuracy skoru
    accuracy = accuracy_score(y_test, y_pred)
    # Bu döngüde elde edilen accuracy skorunu listeye ekle
    accuracy_list.append(accuracy)
    # Bu döngüde kullanılan k değerini listeye ekle
    k_list.append(k)
    # yapılan işi görmek için döngüde kullanılan k, accuracy çiftini ekrana yaz
    print("k {} için accuracy: {:5.2f}".format(k,accuracy))
    
    
    
    

k 2 için accuracy:  0.82
k 3 için accuracy:  0.86
k 4 için accuracy:  0.86
k 5 için accuracy:  0.86
k 6 için accuracy:  0.86
k 7 için accuracy:  0.86
k 8 için accuracy:  0.86
k 9 için accuracy:  0.86


In [40]:
# floating format açıklama:
#  {:5.2f}")
# 5 rakamı bir önceki ifadeden sonra ne kadar boşluk kalsın.
# 2 rakamı ondalık detayı kaç hane olsun.

In [41]:
# Accuracy skorlarının bulunduğu listeden en yüksek skorun indeks numarasını al 
acc_max_index = accuracy_list.index(max(accuracy_list))
acc_max_index

1

In [42]:
# Yukarıdaki indeks k listesinde hangi k sayısına (değere) karşılık geliyorsa
# bu k değeri en iyi sonucu (accuracy) üretiyor.
best_k = k_list[acc_max_index]
print(best_k)

3


## 5.4. En iyi k değeri için modeli eğitelim

In [43]:
classifier = KNeighborsClassifier(n_neighbors=best_k)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

## 5.5. En iyi modelle tahmin yapma

In [44]:
# test setinden tahminler yapalım
y_pred = classifier.predict(X_test)

In [45]:
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: % {:10.2f}".format(accuracy*100)) 
# :10.2f ifadesi ondalık kısımda iki basamak göstermek için

Accuracy: %      86.00


## 5.6. Elle tahmin seti üretme

In [62]:
# 2. indekste bulunan müşteri bilgileri
tahmin_edilecek = ['Erkek',26,43000]

In [63]:
# python listesini numpy'a çevir
tahmin_edilecek_np = np.array(tahmin_edilecek)

In [64]:
tahmin_edilecek_np

array(['Erkek', '26', '43000'], dtype='<U5')

In [65]:
# Şekil kontrolü
tahmin_edilecek_np.shape

(3,)

In [66]:
# Bizim istediğimiz şekilde değil(satır_sayısı, sütun_sayısı) olmalı.
tahmin_edilecek_np = tahmin_edilecek_np.reshape(1,3)

In [67]:
tahmin_edilecek_np.shape

(1, 3)

In [68]:
tahmin_edilecek_np

array([['Erkek', '26', '43000']], dtype='<U5')

In [69]:
# Şimdi istediğimiz şekilde ancak hala model oluştururken yaptığımız işlemleri yapmadık.

In [70]:
tahmin_edilecek_np[:,0]

array(['Erkek'], dtype='<U5')

In [71]:
# Yeni bir label encoder yaratmaya gerek yok. 
label_encoder_cinsiyet.transform(tahmin_edilecek_np[:,0])

array([0], dtype=int64)

In [56]:
tahmin_edilecek_np[:,0] = label_encoder_cinsiyet.transform(tahmin_edilecek_np[:,0])
tahmin_edilecek_np

array([['0', '26', '43000']], dtype='<U5')

In [57]:
tahmin_edilecek_np.shape

(1, 3)

In [58]:
#tahmin_edilecek_np = scaler.fit_transform(tahmin_edilecek_np)
#print(tahmin_edilecek_np.shape)
#print(tahmin_edilecek_np)
#(1, 3)
#[[0. 0. 0.]]

In [59]:
# fit_transform() kullanmamalıyız. Çünkü elimizde hazır eğitilmiş bir scaler var.
tahmin_edilecek_np = scaler.transform(tahmin_edilecek_np)
print(tahmin_edilecek_np.shape)
print(tahmin_edilecek_np)

(1, 3)
[[-1.02020406 -1.11320552 -0.78528968]]


In [60]:
classifier.predict(tahmin_edilecek_np)

array([0], dtype=int64)

In [61]:
# Sonuç satın almaz.